In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("C:/Users/sumit/Downloads/titanic.csv")

In [3]:
data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [ ]:
#Create a new feature: FamilySize = SibSp + Parch + 1

#Create an IsAlone flag (1 if FamilySize == 1, else 0)

#Extract title from passenger names (Mr, Mrs, Miss, etc.)

#Encode categorical variables (Sex, Embarked, Title)

In [28]:
data["FamilySize"] = data["SibSp"] + data["Parch"] +1

data["IsAlone"] = (data["FamilySize"]==1).astype(int)

data['title'] = data['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)

rare_titles = data['title'].value_counts()[data['title'].value_counts() < 10].index
data['title'] = data['title'].replace(rare_titles, 'Rare')

data['title'] = data['title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})

In [29]:
data["title"].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Rare'], dtype=object)

In [30]:
from sklearn.preprocessing import LabelEncoder

In [31]:
le = LabelEncoder()
data["Sex"] = le.fit_transform(data["Sex"])

In [32]:
le1 = LabelEncoder()
data["Embarked"] = le1.fit_transform(data["Embarked"])

In [33]:
le2 = LabelEncoder()
data["title"] = le2.fit_transform(data["title"])

In [34]:
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone,title
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,2,2,0,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0,2,0,3
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,1,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,2,2,0,3
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,2,1,1,2


In [23]:
#for col in ["Sex", "Embarked", "title"]:
    #data[col] = LabelEncoder().fit_transform(data[col])


In [ ]:
#Train Logistic Regression, Random Forest
#Use 5-fold cross-validation and compare average accuracy for each model

In [52]:
new_data = data[["Pclass","Age","Sex","SibSp","Parch","FamilySize","Fare","Embarked","IsAlone","title","Survived"]].copy()

In [53]:
new_data.head(5)

,Pclass,Age,Sex,SibSp,Parch,FamilySize,Fare,Embarked,IsAlone,title,Survived
0,3,22.0,1,1,0,2,7.2500,2,0,2,0
1,1,38.0,0,1,0,2,71.2833,0,0,3,1
2,3,26.0,0,0,0,1,7.9250,2,1,1,1
3,1,35.0,0,1,0,2,53.1000,2,0,3,1
4,3,35.0,1,0,0,1,8.0500,2,1,2,0


In [54]:
new_data.isnull().sum()

Pclass          0
Age           177
Sex             0
SibSp           0
Parch           0
FamilySize      0
Fare            0
Embarked        0
IsAlone         0
title           0
Survived        0
dtype: int64

In [55]:
from sklearn.impute import SimpleImputer

In [56]:
si = SimpleImputer(strategy = "median")
new_data[["Age"]] = si.fit_transform(new_data[["Age"]])

In [57]:
x = new_data.iloc[:,:-1]
y = new_data["Survived"]

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [65]:
lr = LogisticRegression(max_iter = 2000)
lr.fit(x_train,y_train)

LogisticRegression(max_iter=2000)

In [67]:
lr_scores = cross_val_score(lr, x, y, cv=5, scoring='accuracy')
print("Logistic Regression 5-Fold Accuracy:", lr_scores)
print("Logistic Regression Mean Accuracy:", lr_scores.mean())

Logistic Regression 5-Fold Accuracy: [0.78212291 0.78089888 0.80337079 0.7752809  0.83146067]
Logistic Regression Mean Accuracy: 0.7946268281965978


In [69]:
rf = RandomForestClassifier(n_estimators = 8)
rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=8)

In [70]:
rf_scores = cross_val_score(rf, x, y, cv=5, scoring='accuracy')
print("\nRandom Forest 5-Fold Accuracy:", rf_scores)
print("Random Forest Mean Accuracy:", rf_scores.mean())


Random Forest 5-Fold Accuracy: [0.77653631 0.81460674 0.85955056 0.76966292 0.83707865]
Random Forest Mean Accuracy: 0.8114870378507314


In [73]:
results = pd.DataFrame({'Model': ['Logistic Regression', 'Random Forest'],'Mean Accuracy': [lr_scores.mean(), rf_scores.mean()],"5-fold accuracy":[lr_scores, rf_scores]})
results

,Model,Mean Accuracy,5-fold accuracy
0,Logistic Regression,0.794627,"[0.7821229050279329, 0.7808988764044944, 0.803..."
1,Random Forest,0.811487,"[0.776536312849162, 0.8146067415730337, 0.8595..."


In [77]:
#For Logistic Regression: Show the top positive and negative coefficients
coef = lr.coef_[0]
feature_names = x.columns

coef_df = pd.DataFrame({'Feature': feature_names,'Coefficient': coef})
coef_df = coef_df.sort_values(by='Coefficient', ascending=False)

print("Top Positive Coefficients (increase survival probability):")
print(coef_df.head(5))
print("\nTop Negative Coefficients (decrease survival probability):")
print(coef_df.tail(5))

Top Positive Coefficients (increase survival probability):
    Feature  Coefficient
6      Fare     0.002175
4     Parch    -0.008728
1       Age    -0.026439
9     title    -0.101204
7  Embarked    -0.181047

Top Negative Coefficients (decrease survival probability):
      Feature  Coefficient
3       SibSp    -0.225040
5  FamilySize    -0.248445
8     IsAlone    -0.691534
0      Pclass    -0.883035
2         Sex    -2.537883


In [78]:
#For Random Forest : Show top 10 important features
importances = rf.feature_importances_
importance_df = pd.DataFrame({'Feature': feature_names,'Importance': importances}).sort_values(by='Importance', ascending=False)

print("\nTop 10 Important Features in Random Forest:")
print(importance_df.head(10))


Top 10 Important Features in Random Forest:
      Feature  Importance
6        Fare    0.272160
1         Age    0.219338
9       title    0.150135
2         Sex    0.139636
0      Pclass    0.065938
5  FamilySize    0.051055
7    Embarked    0.033895
4       Parch    0.026912
3       SibSp    0.026784
8     IsAlone    0.014147
